In [1]:
try:
    sc.stop()
except:
    pass
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
sc=SparkContext()
spark = SparkSession(sparkContext=sc)

## Funciones agregadas
Dos funciones agregadas:

* `aggregate()`
* `aggregateByKey()`

### `aggregate(zeroValue, seqOp, combOp)`

* **zeroValue** es como un contenedor de datos. Su estructura debe coincidir con la estructura de datos de los valores devueltos por la función seqOp.
* **seqOp** es una función que toma dos argumentos: el primer argumento es el zeroValue y el segundo es un elemento del RDD. El valor cero se actualiza con el valor devuelto después de cada ejecución.
* **combOp** es una función que toma dos argumentos: el primer argumento es el valor cero final de una partición y el otro es otro valor cero final de otra partición.

El siguiente código calcula la suma total de cuadrados para **mpg** y **disp** en el conjunto de datos **mtcars**.

Paso 1: Obtenga algunos datos.

In [2]:
mtcars_df = spark.read.csv('../../data/mtcars.csv', inferSchema=True, header=True).select(['mpg', 'disp'])
mtcars_df.take(5)

[Row(mpg=21.0, disp=160.0),
 Row(mpg=21.0, disp=160.0),
 Row(mpg=22.8, disp=108.0),
 Row(mpg=21.4, disp=258.0),
 Row(mpg=18.7, disp=360.0)]

Paso 2: calcular promedios de mgp y disp

In [3]:
mpg_mean = mtcars_df.select('mpg').rdd.map(lambda x: x[0]).mean()
disp_mean = mtcars_df.select('disp').rdd.map(lambda x: x[0]).mean()
print('mpg mean = ', mpg_mean, '; ' 
      'disp mean = ', disp_mean)

mpg mean =  20.090625000000003 ; disp mean =  230.721875


Paso 3: Construir **zeroValue, seqOp** y **combOp**

Estamos calculando dos TSS. Creamos una tupla para almacenar dos valores.

In [4]:
zeroValue = (0, 0) 

El **z** abajo se refiere a `zeroValue`. Sus valores se actualizan después de cada ejecución. El **x** se refiere a un elemento en una partición RDD. En este caso, tanto **z** como **x** tienen dos valores.

In [5]:
seqOp = lambda z, x: (z[0] + (x[0] - mpg_mean)**2, z[1] + (x[1] - disp_mean)**2)

La función `combOp` simplemente agrega todos los `zeroValues` en uno. 

In [6]:
combOp = lambda px, py: ( px[0] + py[0], px[1] + py[1] )

Implementar la función `aggregate()`.

mtcars_df.rdd.aggregate(zeroValue, seqOp, combOp)

## `aggregateByKey(zeroValue, seqOp, combOp)`

Esta función hace cosas similares a  `aggregate()`. El `aggregate()` agrega todos los resultados hasta el final, pero aggregateByKey() fusiona los resultados por clave.

### Importaar data

In [2]:
iris_rdd = sc.textFile('../../data/iris.csv', use_unicode=True)
iris_rdd.take(2)

['sepal_length,sepal_width,petal_length,petal_width,species',
 '5.1,3.5,1.4,0.2,setosa']

### Transformar data a una tupla RDD

In [3]:
iris_rdd_2 = iris_rdd.map(lambda x: x.split(',')).\
    filter(lambda x: x[0] != 'sepal_length').\
    map(lambda x: (x[-1], [*map(float, x[:-1])]))
iris_rdd_2.take(5)

[('setosa', [5.1, 3.5, 1.4, 0.2]),
 ('setosa', [4.9, 3.0, 1.4, 0.2]),
 ('setosa', [4.7, 3.2, 1.3, 0.2]),
 ('setosa', [4.6, 3.1, 1.5, 0.2]),
 ('setosa', [5.0, 3.6, 1.4, 0.2])]

### Define valores iniciales, seqOp and combOp

In [4]:
zero_value = (0, 0)
seqOp = (lambda x, y: (x[0] + (y[0])**2, x[1] + (y[1])**2))
combOp = (lambda x, y: (x[0] + y[0], x[1] + y[1]))

### Implementa `aggregateByKey()`

In [5]:
iris_rdd_2.aggregateByKey(zero_value, seqOp, combOp).collect()

[('setosa', (1259.0899999999997, 591.2500000000002)),
 ('versicolor', (1774.8600000000001, 388.47)),
 ('virginica', (2189.9000000000005, 447.33))]